In [2]:
# This is its own cell because it takes a while to load this thing
from gensim import models

# takes a lilbit
# model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  

In [3]:
# imports and permanent stuff

import requests
import json
import pickle
import os
import numpy as np
import time
import shutil

In [161]:
## class Loader

class Loader:
    def __init__(self):
        pass
    
    filedir = ""
    list_of_ids = []
    list_of_times = []
    list_of_types = ['text','log','contact','calender','file','tweets']
    
    def deletdatboi(self, nombre):
        shutil.rmtree('./datafor' + list_of_scrape_times[nombre])
    
    def downloadAndLabel(self):
        
        r = requests.get('http://depressionmqp.wpi.edu:8080/getids')
        list_of_idtime = r.json()
        

        for i in range(0,len(list_of_idtime)):
            Loader.list_of_ids.append( list_of_idtime[i]['id'].encode('ascii','ignore') )

        for i in range(0,len(list_of_idtime)):
            Loader.list_of_times.append( list_of_idtime[i]['date'] )

        number_cols = len(Loader.list_of_types)
        number_rows = len(Loader.list_of_ids)
        
        # an artifact of the past
        # list_of_jsons = [[0] * number_cols for i in range(number_rows)]

        
        ### create directory for this particular scrape/pull
        timenow  = str(int(time.time())) # for temporal congruency
        list_of_scrape_times.append(timenow)
        os.mkdir('./datafor' + timenow[-3:])
        Loader.filedir = './datafor' + timenow[-3:]


        for i in range(0,number_rows):
            for j in range(0,number_cols):
                temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(int(Loader.list_of_ids[i])) + '&type=' + Loader.list_of_types[j])

                pickle.dump(temp, open( Loader.filedir + "/DP" + str(int(Loader.list_of_ids[i])) +  Loader.list_of_types[j] + ".p", "wb" ))

                # loads it into memory. i will not use this for now
                # for sake of architectural sanity
                # list_of_jsons[i][j] = temp


In [129]:
## Featurizer featurizes methods that convert json objects of the appropriate type into features

class Featurizer:
    def __init__(self):
        pass
        #self.name = name

    # takes in element of list_of_jsons and scrape date, spits out text frequency
    def dailyTextFreq(self, texts, scrapedate):
        
        a = texts.json()

        seconds_intwo_weeks = 1209600;
        
        # calculate unix time for two weeks prior
        two_weeks_prior = scrapedate - seconds_intwo_weeks

        text_date = 0
        saved_index = 0

        # find the index of the oldest message that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            text_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < text_date):
                saved_index = len(a) - (i+1)
                break

        # divide by 14 to get daily freq
        return float(saved_index+1)/14
    

    # takes in element of list_of_jsons(resp object) and scrape date, spits out call frequency 
    def dailyCallFreq(self, calls, scrapedate):

        a = calls.json()

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        call_date = 0
        saved_index = 0

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            call_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < call_date):
                saved_index = len(a) - (i+1)
                break

        return float(saved_index+1)/14
    
    
    #input is tweets, return master vector
    def embeddingToMastersum(self, tweets):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,10):
            try:
                masterSum += self.tweetToEmbedding(tweets.json()[i])
            except:
                pass
        
        return masterSum

    
    # input is tweet, returns vector embedding of entire tweet.
    # eg: responseobject.json()[0]
    def tweetToEmbedding(self, tweet):

        q = json.loads(tweet)['text'].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    #input is tweets, return follocount
    def followerCount(self, tweets):

        followerCount = 0

        # get
        try:
            followerCount = json.loads(tweets.json()[0])['user']['followers_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followerCount

    #input is tweets, return friends (who user follows)
    def followingCount(self, tweets):

        followingCount = 0

        try:
            followingCount = json.loads(tweets.json()[0])['user']['friends_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followingCount

    #input is tweets, return avg likes per post for the last 2 weeks
    def twitterLikeFreq(self, tweets, scrapedate):

        a = tweets.json()

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_likes = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets.json()[i])['favorite_count']


        return sum_of_likes/14

    #input is tweets, return avg retweets per post for the last 2 weeks
    def twitterRetweetFreq(self, tweets, scrapedate):

        a = tweets.json()

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_retweets = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets.json()[i])['retweet_count']


        return sum_of_retweets/14
    
    # input is contacts, returns number of contacts
    def numOfContacts(self, contacts):
        return len(contacts.json())
    


In [169]:
class Generator:

    def __init__(self, filedir, list_of_ids):
        self.filedir = filedir
        self.list_of_ids = list_of_ids
    
    featureVectorCF = np.zeros((num_of_people,1))
    featureVectorTF = np.zeros((num_of_people,1))
    featureVectorTW = np.zeros((num_of_people,300))
    featureVectorFC = np.zeros((num_of_people,1))
    featureVectorFC2 = np.zeros((num_of_people,1))
    featureVectorTWL = np.zeros((num_of_people,1))
    featureVectorTWRT = np.zeros((num_of_people,1))
    featureVectorNC = np.zeros((num_of_people,1))
    featureMatrix = None
    
    f = Featurizer()

    def generateMatrix(self):
        
        num_of_people = len(list_of_ids)

        for i in range(0, num_of_people):

            #list_of_jsons[i][1]

            a1 = pickle.load( open( self.filedir + "/DP" + str(int(list_of_ids[i])) +  "log" + ".p", "rb" )) 
            a2 = pickle.load( open( self.filedir + "/DP" + str(int(list_of_ids[i])) +  "text" + ".p", "rb" )) 
            a3 = pickle.load( open( self.filedir + "/DP" + str(int(list_of_ids[i])) +  "tweets" + ".p", "rb" )) 
            a4 = pickle.load( open( self.filedir + "/DP" + str(int(list_of_ids[i])) +  "contact" + ".p", "rb" )) 


            Generator.featureVectorCF[i] = f.dailyCallFreq(a1, list_of_times[i])
            Generator.featureVectorTF[i] = f.dailyTextFreq(a2, list_of_times[i])
            Generator.featureVectorTW[i] = f.embeddingToMastersum(a3)
            Generator.featureVectorFC[i] = f.followerCount(a3)
            Generator.featureVectorFC2[i] = f.followingCount(a3)
            Generator.featureVectorTWL[i] = f.twitterLikeFreq(a3, list_of_times[i])
            Generator.featureVectorTWRT[i] = f.twitterRetweetFreq(a3, list_of_times[i])
            Generator.featureVectorNC[i] = f.numOfContacts(a4)
            

            Generator.featureMatrix = np.hstack((featureVectorCF,featureVectorTF,
featureVectorTW, featureVectorFC, featureVectorTWL, featureVectorTWRT))


In [170]:
# Everything I coded in one cell!

#l = Loader()
#l.downloadAndLabel()

g = Generator(l.filedir, l.list_of_ids)
g.generateMatrix()
g.featureMatrix.shape

(53, 305)

In [99]:
## WASTELAND OF PLAY CELLS BELOW


filedir = './datafor' + '1511239205'

## access example
a3 = pickle.load( open( filedir + "/DP" + str(int(list_of_ids[2])) +  "contact" + ".p", "rb" )) 
# json.loads(a3.json()[0])
# a3.json()
list_of_ids[0]

len(a3.json())





2

In [143]:
w = embeddingToMastersum(a3)
w.reshape((-1, 1)).T.shape


(1, 300)

In [156]:
timenow  = str(int(time.time())) # for temporal congruency
timenow1 = timenow
timenow1

'1511765588'

In [159]:
timenow[-3:]

'588'